In [2]:
import numpy as np
import argparse
import os
import json
import time
import random
import tqdm

from rl_gomoku.envs.array_gomoku import ArrayGomoku
from rl_gomoku.utils import start_play, collect_data, start_self_play, ModelInterface, collect_self_play_data, replay_data_from_list
from rl_gomoku.agents.mcts_alpha import MCTSZeroAgent
from rl_gomoku.agents import GreedyAgent
from rl_gomoku.envs import GomokuEnv
from collections import deque

In [3]:
env = ArrayGomoku()

In [1]:
import os

from cache_data.utils import best_bots

In [9]:
def generate_unique_months_for_year(year):
    return [f"{year}-{month}" for month in range(1, 13)]

# Generate months for 2021 to 2023
unique_months_2021_to_2023 = generate_unique_months_for_year(2021) + generate_unique_months_for_year(2022) + generate_unique_months_for_year(2023)
unique_months_2021_to_2023 = generate_unique_months_for_year(2022) + generate_unique_months_for_year(2023)
unique_months_2021_to_2023 = unique_months_2021_to_2023[6:-1]

In [12]:
lines = []
for date in unique_months_2021_to_2023:
    _dir = f'./cache_data/Gomoku-{date}/'
    matches = os.listdir(_dir)
    for x in matches:
        with open(f"{_dir}/{x}", "r") as f:
            lines.extend(f.readlines())

In [13]:
def parse_data(line, best_bots):
    result = {}
    try:
        player_0 = line['players'][0]
        player_1 = line['players'][1]
        
        if player_0["type"] == "bot" and player_1["type"] == "bot":
            result["type"] = "bot"
            result["bot_0"] = player_0["bot"]
            result["bot_1"] = player_1["bot"]
        else:
            result["type"] = "browser"
            result["bot_0"] = None
            result["bot_1"] = None
        if result["bot_0"] in best_bots or result["bot_1"] in best_bots:
            result["valid"] = True
        else:
            result['valid'] = False
            
        if result["valid"]:
            tmp = list(filter(lambda x: "0" in x or "1" in x, line['log']))
            tmp = [list(x.values())[0]["response"] for x in tmp]
            to_position = lambda x: x["x"] * 15 + x["y"]
            tmp = [to_position(x) for x in tmp]
            result["len"] = len(tmp)
            result["data"] = tmp
            result["score"] = line["scores"]
        return result
    except KeyError:
        return {"valid": False}

In [16]:
import joblib
from joblib import delayed, Parallel
import json

In [17]:
works_to_do = [delayed(json.loads)(line) for line in lines]
lines = Parallel(n_jobs=20)(works_to_do)

In [ ]:
# works_to_do = [delayed(parse_data)(line, best_bots[:10]) for line in lines]
# line_dict = Parallel(n_jobs=16)(works_to_do)

In [18]:
line_dict = [parse_data(line, best_bots[:7]) for line in lines]

In [19]:
true_dict = [ld for ld in line_dict if ld["valid"]]

In [20]:
len(true_dict)

8895

In [4]:
import joblib

In [22]:
valid_dict = joblib.load("./cache_data/botzone.joblib")

In [23]:
len(valid_dict)

8895

In [55]:
winner, *_ = replay_data_from_list(env, true_dict[3]["data"], render=True)

   0  1  2  3  4  5  6  7  8  9  10 11 12 13 14
0  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
1  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
2  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
3  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
4  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
5  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
6  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
7  _  _  _  _  _  _  _  X  _  _  _  _  _  _  _
8  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
9  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
10 _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
11 _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
12 _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
13 _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
14 _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
last position: 112, player: 2
   0  1  2  3  4  5  6  7  8  9  10 11 12 13 14
0  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
1  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
2  _  _  _  _  _  _  _  _  _  _  _  _  _  _  _
3  _  _  _  _  _  _  _  _  _